In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Reading in datasets

In [ ]:
control_visits = pd.read_csv("rw1276_telehealth_controls_visits.csv",encoding='latin-1')
df_control_visits = control_visits[(control_visits["encounter_status_name"]=="Completed")|
                                   (control_visits["encounter_status_name"]=="No Show")|
                                  (control_visits["encounter_status_name"]=="Canceled")]

cases_visits = pd.read_csv("rw1276_telehealth_cases_visits.csv",encoding='latin-1')
df_cases_visits = cases_visits[(cases_visits["encounter_status_name"]=="Completed")|
                               (cases_visits["encounter_status_name"]=="No Show")|
                              (cases_visits["encounter_status_name"]=="Canceled")]

df_all = pd.concat([df_control_visits,df_cases_visits])
#only keeping the Illinois zipcode
df_all2 = df_all[df_all["patient_postal_code"].str.startswith(("60","61","62"))==True]
#f_all2.to_csv("~/Data/telehealth_combined_ILonly_221222.csv")

In [ ]:
#Removing Peds 
df_combined_IL2 = df_all2[df_all2["provider_specialty"].str.contains("Child|CHILD|child|Pedi|pedi") == False]
df_combined_IL3 = df_combined_IL2[df_combined_IL2["department_specialty"].str.contains("Child|CHILD|child|Pedi|pedi") == False]
df = df_combined_IL3.copy()
df["timepoint"]= df["encounter_start_date_key"].apply(lambda x: x[0:4])
df = df[(df["timepoint"]=="2019")|
         (df["timepoint"]=="2020")|
         (df["timepoint"]=="2021")|
         (df["timepoint"]=="2022")]

df = df[(df['patient_state_province']=="Illinois") | (df['patient_state_province']=="IL")]

In [ ]:
#race ethnicity 
df.loc[df.race_1.isnull()==True, "race_1"]="Unknown"
df.loc[df.ethnic_group.isnull()==True, "ethnic_group"]="Unknown"
race_map = {'White': 'White',
                   'Black or African American': 'Black',
                   'Other': 'Other',
                   'Declined': 'Other',
                   'Declined to answer': 'Other',
                   'Asian': 'Other',
                   'Unable to Answer': 'Other',
                   'Unknown': 'Other',
                   'American Indian or Alaska Native':'Other',
                   'Native Hawaiian or Other Pacific Islander':'Other',
                   '2 or more races': 'Other',
                   'Hispanic': 'Hispanic',
                   'American Indian or Alaskan Native':'Other'}

df['race_cat'] = df['race_1'].map(race_map)

ethnic_map = {'Not Hispanic or Latino': 'Non-Hispanic',
                   'Hispanic or Latino': 'Hispanic',
                   'Declined': 'Declined to answer',
                   'Unable to Answer': 'Other',
                     'Unknown': 'Other'}

df['ethnic_cat'] = df['ethnic_group'].map(ethnic_map)

df["race_ethnic"] = df["ethnic_cat"] + " " + df["race_cat"]

race_ethnic_map = {'Non-Hispanic White': 'Non-Hispanic White',
                   'Non-Hispanic Black': 'Non-Hispanic Black',
                   'Other Other': 'Other',
                   'Hispanic White': 'Hispanic',
                   'Non-Hispanic Other': 'Other',
                  'Hispanic Other': 'Hispanic',
                  'Non-Hispanic Asian': 'Asian',
                  'Other White': 'Other',
                  'Other Black':'Other',
                  'Other Asian':'Asian',
                  'Hispanic Black':'Hispanic',
                  'Hispanic Asian':'Asian',
                  'Declined to answer Declined to answer':'Declined to answer',
                  'Declined to answer White': 'Declined to answer',
                  'Non-Hispanic Declined to answer':'Declined to answer',
                  'Declined to answer Other':'Declined to answer',
                  'Hispanic Declined to answer':'Hispanic',
                  'Declined to answer Black':'Declined to answer',
                  'Declined to answer Asian':'Asian',
                  'Other Declined to answer':'Declined to answer'}

df['race_ethnic_cat'] = df['race_ethnic'].map(race_ethnic_map)

In [ ]:
#economic distress
median_income = pd.read_csv("median_income.csv")
median_income = median_income[["NAME","S1901_C01_012E"]]
median_income2 = median_income.iloc[1: , :]
median_income2 = median_income2.rename(columns = {"NAME": "Zipcode", "S1901_C01_012E":"Household Median Income"})
median_income2["Zipcode"] = median_income2["Zipcode"].str[-5:]
list_zip_dropped = list(median_income2[median_income2["Household Median Income"]=="-"]["Zipcode"])
median_income2 = median_income2[median_income2["Household Median Income"]!="-"]
median_income2["Household Median Income"] = median_income2["Household Median Income"].astype('int')
median_income3 = median_income2.sort_values("Household Median Income",ascending=True)
median_income_lowest25 = median_income3.iloc[0: 335, :]
df.loc[df.primary_financial_class.isnull()==True, "primary_financial_class"]="Other"

insurance_map = {'BCBS': 'Commercial',
                   'Commercial/Managed Care': 'Commercial',
                   'Medicare': 'Medicare',
                   'Self-Pay': 'Self-Pay',
                   'Medicaid Replacement': 'Medicaid',
                  'Medicare Advantage': 'Medicare Advantage',
                  'Self-pay': 'Self-Pay',
                  'Medicaid': 'Medicaid',
                  'Non-Contracted/Commercial':'Commercial',
                  "Worker's Comp":'Other',
                  'Pending Medicaid':'Medicaid',
                'Commercial':'Commercial',
                'Medicare Advantage/Replacement':'Medicare Advantage',
                'Other':'Other'}

df['insurance_cat2'] = df['primary_financial_class'].map(insurance_map)
df['insurance_cat2'].value_counts(dropna=False)
df = df[df["insurance_cat2"]!="Other"]
list_lowest25 = list(median_income_lowest25["Zipcode"])
df["is_lowest25"] = df["patient_postal_code"].isin(list_lowest25)
df["zip_dropped"] = df["patient_postal_code"].isin(list_zip_dropped)
df["patient_postal_code"] = df["patient_postal_code"].str[:5]
df["economic_distress_cat"] = df.apply(lambda x: "HPED" if (((x["insurance_cat"]=="Medicare")|(x["insurance_cat"]=="Medicaid")) 
                               & x["is_lowest25"]==True)
                                       else ("dropped zip" if (x["zip_dropped"]==True)
                                           else "LPED"), axis=1)
df2 = df[df['economic_distress_cat']!="dropped zip"]


In [ ]:
#age
age_cut_labels = ['18-39', '40-64','65+']
age_cut_bins = [0, 39, 64,150]
df['age_cat'] = pd.cut(df['age_at_visit'], bins=age_cut_bins, labels=age_cut_labels)
df['age_cat'].value_counts(dropna=False)

#sex
df = df[df["gender"]!="U"]

#comorbidity 
Charlson_cut_labels = ['0','1','2','3+']
Charlson_cut_bins = [-1, 0, 1, 2, 100]
df3['Charlson_cat'] = pd.cut(df['Charlson_score'], bins=Charlson_cut_bins, labels=Charlson_cut_labels)
df3['Charlson_cat'].value_counts(dropna=False)

In [ ]:
#additional cleanup 
median_income = pd.read_csv("/share/fsmresfiles/telehealth/median_income.csv")
median_income = median_income[["NAME","S1901_C01_012E"]]
median_income2 = median_income.iloc[1: , :]
median_income2 = median_income2.rename(columns = {"NAME": "Zipcode", "S1901_C01_012E":"Household Median Income"})
median_income2["Zipcode"] = median_income2["Zipcode"].str[-5:]
median_income2[median_income2["Household Median Income"]=="-"].shape
median_income2 = median_income2[median_income2["Household Median Income"]!="-"]
median_income2["Household Median Income"] = median_income2["Household Median Income"].astype('int')
median_income3 = median_income2.copy()
median_income3["Median_Income_Cat"] = median_income3["Household Median Income"].apply(lambda x:
                                                                                     "Q1" if(x<51194)
                                                                                      else ("Q2" if ((x>=51194)& (x<61710))
                                                                                            else ("Q3" if ((x>=61710) & (x<76255))
                                                                                                 else "Q4")))
df["patient_postal_code"] = df["patient_postal_code"].astype(str)
median_income3["Zipcode"] = median_income3["Zipcode"].astype(str)
df = df.merge(median_income4, left_on="patient_postal_code", right_on = "Zipcode", how='left')
df = df.dropna(subset=['Zipcode'])

specialty_list = [ "Internal Medicine",
                  "Family Medicine",
                "Obstetrics and Gynecology",
                  "Gynecology",
                 "Cardiology: Invasive Interventional",
                 "Preventive Cardiology",
                 "Cardiology: Electrophysiology",
                 "Cardiology",
                 "Interventional Cardiology",
                 "Cardiology: Noninvasive",
                 "Dermatology",
                  "Psychiatry", 
                  "Psychology", 
                  "Behavioral Health",
                  "Gastrointestinal Behavioral Health Psychology",
                  "PSYCH-BEHAVIORAL",
                  "Sleep Psychologist",
                  "Psychiatry -- Non MD",
                  "PSYCH-BEHAVIORAL",
                  "Psychosomatic Medicine", 
                  "Psych-Clinical",
                  "Geriatric Psychiatry",
                 "Addiction Psychiatry", 
                  "PSYCHIATRY-ADDICTION",
                  "COUNSELOR",
                  "Neuropsychology",
                  "Social Work"
                 "Dietary Medicine",
                 "Endocrinology",
                 "Reproductive Endocrinology and Infertility",
                 "Bariatric Endocrinology",
                 "Pulmonology",
                 "Interventional Pulmonology",
                 "Pulmonary Disease",
                 "Reproductive Genetics",
                 "Genetic",
                 "Genetic Counseling",
                 "Medical Genetics",
                 "Hematology and Medical Oncology",
                 "Radiation Oncology",
                 "Medical Oncology",
                 "Urologic Oncology",
                 "Neuro-Oncology",
                 "Orthopaedic Oncology",
                 "Medical Oncology with Infusion",
                 "Hematology/Oncology",
                 "Oncology",
                 "NeuroOncology"]
df = df[df["provider_specialty"].isin(specialty_list)]

def label_reasons (row):
    row["provider_specialty"] = str(row["provider_specialty"]).strip()
    if row["provider_specialty"] in ["Internal Medicine",
                  "Family Medicine"]:
        return "Primary Care"
    elif row["provider_specialty"] in ["Obstetrics and Gynecology",
                  "Gynecology"]:
        return "OGBYN"
    elif row["provider_specialty"] in ["Psychiatry", 
                  "Psychology", 
                  "Behavioral Health",
                  "Gastrointestinal Behavioral Health Psychology",
                  "PSYCH-BEHAVIORAL",
                  "Sleep Psychologist",
                  "Psychiatry -- Non MD",
                  "PSYCH-BEHAVIORAL",
                  "Psychosomatic Medicine", 
                  "Psych-Clinical",
                  "Geriatric Psychiatry",
                 "Addiction Psychiatry", 
                  "PSYCHIATRY-ADDICTION",
                  "COUNSELOR",
                  "Neuropsychology",
                  "Social Work",
                  "Dietary Medicine"]:
        return "Mental Health and Behavioral Health"
    elif row["provider_specialty"] in [
                "Cardiology: Invasive Interventional",
                "Preventive Cardiology",
                "Cardiology: Electrophysiology",
                 "Cardiology",
                 "Cardiology",
                 "Interventional Cardiology",
                 "Cardiology: Noninvasive"]:
        return "Cardiology"
    elif row["provider_specialty"] in ["Endocrinology",
                 "Reproductive Endocrinology and Infertility",
                 "Bariatric Endocrinology",
                 "Endocrine Surgery",]:
        return "Endocrinology"
    elif row["provider_specialty"] in ["Orthopaedic Surgery",
                 "Orthopaedic Sports Medicine"]:
        return "Orthopedics"
    elif row["provider_specialty"] in ["Urology",
                 "Genitourinary Reconstructive Surgery"]:
        return "Urology"
    elif row["provider_specialty"] in ["Pulmonology",
                 "Interventional Pulmonology",
                 "Pulmonary Disease"]:
        return "Pulmonology"
    elif row["provider_specialty"] in ["Reproductive Genetics",
                 "Genetic",
                 "Genetic Counseling",
                 "Medical Genetics"]:
        return "Genetics"
    elif row["provider_specialty"] in ["Hematology and Medical Oncology",
                 "Radiation Oncology",
                 "Medical Oncology",
                 "Urologic Oncology",
                 "Neuro-Oncology",
                 "Surgical Oncology",
                 "Neurosurgical Oncology",
                 "Orthopaedic Oncology",
                 "Medical Oncology with Infusion",
                 "Hematology/Oncology",
                 "Oncology",
                 "NeuroOncology"]:
        return "Oncology"
 
    return row["provider_specialty"]
df["provider_specialty_cat"]=df.apply(lambda row: label_reasons(row), axis=1)

df = df[df["encounter_type_name"]!="Erroneous Encounter"]
df = df[df["encounter_type_name"]!="Erroneous Telephone Encounter"]
df["encounter_type_name"].value_counts()

In [ ]:
df["year_month"] = df["encounter_start_date_key"].str[0:7]
df2 = df[(df["year_month"]!="2022-07")&(df["year_month"]!="2022-08")]
conditions = [
    (df2['year_month'] == "2019-01"),
    (df2['year_month'] == "2019-02"),
    (df2['year_month'] == "2019-03"),
    (df2['year_month'] == "2019-04"),
    (df2['year_month'] == "2019-05"),
    (df2['year_month'] == "2019-06"),
    (df2['year_month'] == "2019-07"),
    (df2['year_month'] == "2019-08"),
    (df2['year_month'] == "2019-09"),
    (df2['year_month'] == "2019-10"),
    (df2['year_month'] == "2019-11"),
    (df2['year_month'] == "2019-12"),
    (df2['year_month'] == "2020-01"),
    (df2['year_month'] == "2020-02"),
    (df2['year_month'] == "2020-03"),
    (df2['year_month'] == "2020-04"),
    (df2['year_month'] == "2020-05"),
    (df2['year_month'] == "2020-06"),
    (df2['year_month'] == "2020-07"),
    (df2['year_month'] == "2020-08"),
    (df2['year_month'] == "2020-09"),
    (df2['year_month'] == "2020-10"),
    (df2['year_month'] == "2020-11"),
    (df2['year_month'] == "2020-12"),
     (df2['year_month'] == "2021-01"),
    (df2['year_month'] == "2021-02"),
    (df2['year_month'] == "2021-03"),
    (df2['year_month'] == "2021-04"),
    (df2['year_month'] == "2021-05"),
    (df2['year_month'] == "2021-06"),
    (df2['year_month'] == "2021-07"),
    (df2['year_month'] == "2021-08"),
    (df2['year_month'] == "2021-09"),
    (df2['year_month'] == "2021-10"),
    (df2['year_month'] == "2021-11"),
    (df2['year_month'] == "2021-12"),
     (df2['year_month'] == "2022-01"),
    (df2['year_month'] == "2022-02"),
    (df2['year_month'] == "2022-03"),
    (df2['year_month'] == "2022-04"),
    (df2['year_month'] == "2022-05"),
    (df2['year_month'] == "2022-06")]

values=["Jan to Mar 2019",
       "Jan to Mar 2019",
       "Jan to Mar 2019",
       "Apr to Jun 2019",
       "Apr to Jun 2019",
       "Apr to Jun 2019",
       "Jul to Sep 2019",
       "Jul to Sep 2019",
       "Jul to Sep 2019",
        "Oct to Dec 2019",
       "Oct to Dec 2019",
       "Oct to Dec 2019",
       "Jan to Mar 2020",
       "Jan to Mar 2020",
       "Jan to Mar 2020",
       "Apr to Jun 2020",
       "Apr to Jun 2020",
       "Apr to Jun 2020",
       "Jul to Sep 2020",
       "Jul to Sep 2020",
       "Jul to Sep 2020",
        "Oct to Dec 2020",
       "Oct to Dec 2020",
       "Oct to Dec 2020",
        "Jan to Mar 2021",
       "Jan to Mar 2021",
       "Jan to Mar 2021",
       "Apr to Jun 2021",
       "Apr to Jun 2021",
       "Apr to Jun 2021",
       "Jul to Sep 2021",
       "Jul to Sep 2021",
       "Jul to Sep 2021",
        "Oct to Dec 2021",
       "Oct to Dec 2021",
       "Oct to Dec 2021",
        "Jan to Mar 2022",
       "Jan to Mar 2022",
       "Jan to Mar 2022",
       "Apr to Jun 2022",
       "Apr to Jun 2022",
       "Apr to Jun 2022"]

df2['month_quarter'] = np.select(conditions, values)

#distinguishing phoen and vieo calls 
df3 = df2.copy()
df3["video_vs_tele"] = pd.np.where(df3.telehealth_visit_type.str.contains("VIDEO"), "video",
                   pd.np.where(df3.telehealth_visit_type.str.contains("PHONE"), "phone",
                   pd.np.where(df3.telehealth_visit_type.str.contains("TELEPHONE"), "phone","neither")))

df3 = df3[df3["video_vs_tele"]!="neither"]
df3 = df3[df3["video_vs_tele"]=="video"]